<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/summarization/Langchain_T5_base_Document_summarizer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization

Summarization creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. Summarization can be:

Extractive: extract the most relevant information from a document.
Abstractive: generate new text that captures the most relevant information.

https://huggingface.co/docs/transformers/tasks/summarization

https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/5%20Levels%20Of%20Summarization%20-%20Novice%20To%20Expert.ipynb

https://python.langchain.com/docs/modules/data_connection/document_loaders/


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.0 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install sentence-transformers spacy  trl datasets pypdf -qqq --progress-bar off
!pip install  -U langchain==0.0.309 --quiet

In [ ]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader

# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model


# Embedding Support
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans


# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import transformers
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests

response = requests.get("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

In [ ]:
book_complete_text = response.text

In [ ]:
book_complete_text = book_complete_text[5:]

In [ ]:
len(book_complete_text)

In [ ]:
file_path = "/content/drive/MyDrive/data/book.txt"

In [ ]:
with open(file_path, "w", encoding="utf-8") as f:
    f.write(book_complete_text)

In [ ]:

with open(file_path, "r",  encoding="utf-8") as f:
    text = f.read()

In [ ]:
text = text.replace('\t', ' ')

In [ ]:
len(text)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=500)



In [ ]:
docs = text_splitter.create_documents([text])
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

In [ ]:
text_summarization_pipeline = transformers.pipeline("summarization", model="olonok/olonok_billsum_model")


llm = HuggingFacePipeline(pipeline=text_summarization_pipeline )

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type="map_reduce",
                                     verbose=True)

In [ ]:
output = summary_chain.run(docs) # 13 mins

In [ ]:
print(output.strip())

In [ ]:
len(output.strip())

In [ ]:
model_name = "sentence-transformers/gtr-t5-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)




vectors = embeddings.embed_documents([x.page_content for x in docs])

In [ ]:
len(vectors[0])

In [ ]:
num_clusters = int(len(vectors) // 4)
num_clusters

In [ ]:
# Assuming 'embeddings' is a list or array of 768-dimensional embeddings

# Choose the number of clusters, this can be adjusted based on the book's content.
# I played around and found ~10 was the best.
# Usually if you have 10 passages from a book you can tell what it's about
num_clusters = 5 if num_clusters <=5 else num_clusters

# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)

In [ ]:
kmeans.labels_

In [ ]:
len(kmeans.labels_)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
# Taking out the warnings
import warnings
from warnings import simplefilter

# Filter out FutureWarnings
simplefilter(action='ignore', category=FutureWarning)

# Perform t-SNE and reduce to 2 dimensions
tsne = TSNE(n_components=2, random_state=42)
reduced_data_tsne = tsne.fit_transform(np.array(vectors))

# Plot the reduced data
plt.scatter(reduced_data_tsne[:, 0], reduced_data_tsne[:, 1], c=kmeans.labels_)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Book Embeddings Clustered')
plt.show()

In [ ]:
# Find the closest embeddings to the centroids

# Create an empty list that will hold your closest points
closest_indices = []

# Loop through the number of clusters you have
for i in range(num_clusters):

    # Get the list of distances from that particular cluster center
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)

    # Find the list position of the closest one (using argmin to find the smallest distance)
    closest_index = np.argmin(distances)

    # Append that position to your closest indices list
    closest_indices.append(closest_index)

In [ ]:
selected_indices = sorted(closest_indices)
selected_indices

In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import transformers
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
selected_docs = [docs[doc] for doc in selected_indices]

In [ ]:
selected_docs[0]

In [ ]:
output_reduce = summary_chain.run(selected_docs)

In [ ]:
print(output_reduce.strip())

In [ ]:
len(output_reduce.strip())

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="olonok/olonok_billsum_model")

In [ ]:
selected_docs[1].page_content

In [ ]:
response = summarizer(selected_docs[1].page_content)
response[0]['summary_text']

In [ ]:
# Make an empty list to hold your summaries
summary_list = []

# Loop through a range of the lenght of your selected docs
for i, doc in enumerate(selected_docs):

    # Go get a summary of the chunk
    #chunk_summary = map_chain.run([doc])
    response = summarizer(doc.page_content)
    chunk_summary = response[0]['summary_text']
    # Append that summary to your list
    summary_list.append(chunk_summary)

    print (f"Summary #{i} (chunk #{selected_indices[i]}) - Preview: {chunk_summary[:500]} \n")

In [ ]:
summaries = "\n".join(summary_list)

# Convert it back to a document
summaries = Document(page_content=summaries)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

In [ ]:
print(summaries.page_content)

In [ ]:
len(summaries.page_content)

In [ ]:
text_final = summaries.page_content
docsf = text_splitter.create_documents([text_final])
num_documents = len(docsf)

print (f"Now our book is split up into {num_documents} documents")

In [ ]:
# Make an empty list to hold your summaries
summary_final = []

# Loop through a range of the lenght of your selected docs
for i, doc in enumerate(docsf):

    # Go get a summary of the chunk
    #chunk_summary = map_chain.run([doc])
    response = summarizer(doc.page_content)
    chunk_summary = response[0]['summary_text']
    # Append that summary to your list
    summary_final.append(chunk_summary)

    print (f"Summary #{i} (chunk #{selected_indices[i]}) - Preview: {chunk_summary[:500]} \n")

In [ ]:
summary_final = "\n".join(summary_final)

# Convert it back to a document
summaries = Document(page_content=summary_final)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

In [ ]:
print(summaries.page_content)

In [ ]:
len(summaries.page_content)